# From: Transformers and Text-Generation
by Liam Dugan (UPenn). 


Please write your answers and code in the cells with questions below. 

----------

For this homework, we will take ideas from the entire class: language models, text generation, vector-based word representations, syntactic analysis, and neural networks. We'll be using large, pre-trained language models to generate text, and studying how we can fine-tune these large language models to generate text in whatever genre and style we want!

In this assignment you will get:
1. An overview of the "Transformer" architecture is and why it is particularly well suited for Natural Language Processing tasks
2. An introduction to the Generative Pretrained Transformer (GPT) family, which is a set of large-scale language models that can be used to generate text that often sounds like it was written by a human.
3. Experience with using the HuggingFace package to fine-tune these models to generate text that sounds like it comes from a specific source.

# Part 1: What is a Transformer? (Reading)
<figure align="center">
<img src="https://media.giphy.com/media/VeWllmR9zfaco/giphy.gif" />
<figcaption>(It's probably not this guy, right?)</figcaption>
</figure>

### The Transformer

The current state-of-the-art for a variety of natural language processing tasks belongs to the **Transformer** architecture, first published December 6th 2017. 

The Transformer can be thought of as a big feed-forward network with every feed-forward layer containing something called an "attention module". 

>You might be wondering: why are we moving back to feed-forward networks after having so much success with recurrent neural networks and variants like LSTMs? Aren't RNNs naturally poised to handle sequences as their inputs? Well, as it turns out, the sequential nature of RNNs make them really difficult to train in a distributed/parallel fashion. So while RNNs make more sense to use on sequences of inputs, serial networks such as the transformer can be trained much faster, allowing orders of magnitude more training data to be used. 



### Reading \# 1 - What is a Transformer?

In order to get a good grasp on exactly *why* these models are so good it's important to understand what they are and how they work. 

Your first task for this homework is to read the blog post ["The Illustrated Transformer" by Jay Alammar](http://jalammar.github.io/illustrated-transformer/). This blog post explains the transformer architecture (and the all-important "Attention Module") with helpful visualizations and diagrams. 

**You should read this post very closely and understand exactly what the Transformer is and how it works. Once you're finished reading, answer the following questions in 2-3 sentences each.**

1. (2 pts) What is Self-Attention (at a high level)?

<u>Self-Attention layer is needed to process each one of the word embeddings in input to generate a new vector representation, for each word, that takes into account the connection with all the other words in the sentence we are encoding. In this way we are able to bind specific words that are connected grammatically in the sentence, and represent this connection in the new vector representation the self-attention layer produce.</u>

2. (2 pts) How is Self-Attention computed?

<u>The self attention is computed in a multiple steps process as follow:
   - Product of each word embedding with the Query, Key and Value matrices. We obtain queries, keys and values vectors representation for each word.
   - Compute the score for each words as the product of the query vector from the previous word and key vector from the current word. 
   - Divide the scores by 8 and normalize by softmax.
   - Multiply the value vector with the corresponding softmax coefficient.
   - Sum up the weighted value vectors.</u>


3. (2 pts) What do the "Query", "Key", and "Value" vectors encode (at a high level)?

<u> The vectors are computed for each word and they are abstractions useful for calculating the attention coeficient for that word in relation to other words vectors key and query. 
- Query: The query is a representation of the current word used to score against all the other words.
- Key: like labels for all the words in the segment. They’re what we match against in our search for relevant words.
- Value: Value vectors are actual word representations, once we’ve scored how relevant each word is, these are the values we add up to represent the current word.</u>
   
4. (2 pts) What is an attention "head" and why should we use multiple heads?

<u>An attention head is represented by one group of query, key and value matrices, needed to compute the vectors representations of the self-attention module for the words in input. Instead if we want to compute multiple representations to allow more flexibility to the model we can add multiple attention heads, and this means adding multiple weights matrices (queries, keys and values).</u>

5. (2 pts) What are positional embeddings?

<u>Positional embeddings are additional vectors that keep track of the distance between words in the sentence, or in other words, the temporal position in the sentence. We add the positional embedding to the input word vector before feeding it to the encoder layer (self-attention layer etc...).</u>

6. (2 pts) Why are positional embeddings important?

<u>They are important because they add an additional information to the embedding vectors resulting from the self-attention layer. This information is related, as said before, to the time position of the word in the sequence.</u>




-----------
### Reading \# 2 - Transformer Language Models

On June 11th 2018, OpenAI released a model named **GPT**, standing for *Generative Pre-Trained Transformer*. This model was a Transformer architecture that was modified such that it could be used for Text Generation instead of sequence to sequence modeling. This model was also pre-trained, which means that anyone could download the fully trained model and use it without needing to train the model themselves. 

On February 14th 2019, OpenAI released a blog post detailing a brand new version of GPT that had an insane **1.5 billion parameters**. They named this version **GPT-2**. To train such a large model, OpenAI crawled 40GB worth of text from the web (roughly 20,000,000,000 words). 

GPT-2 is an extremely impressive language model that can generate text that often sounds so plausible that it seems like it might have been written by a human.  Here is an example of what GPT-2 can generate, taken from [OpenAI’s blog post](https://openai.com/blog/better-language-models/):

**Human-Written Prompt:**
> “Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today.”

**Model Continuation:**
> The 19-year-old singer was caught on camera being escorted out of the store by security guards.
> 
> The singer was wearing a black hoodie with the label ‘Blurred Lines’ on the front and ‘Fashion Police’ on the back.
> 
> Scroll down for video
> 
> Shoplifting: Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today (pictured)
> 
> The singer was also wearing a pair of black-rimmed glasses, a black jacket, black jeans and black sandals.
> 
> She was carrying a pair of black and white striped gloves and a small black bag.

*(To be clear: Miley Cyrus was never actually arrested for shoplifting)*.  

The quality of the model's output was so good, that the researchers at OpenAI were worried that, in addition to positive applications, it might be used for malicious purposes like generating fake news, impersonating people online, creating abusive or fake content to post on social media, or automating spam and phishing scams. In a [demo to Wired Magazine](https://www.wired.com/story/ai-text-generator-too-dangerous-to-make-public/), the reporter typed in “Hillary Clinton and George Soros” and GPT-2 generated a crazy rant with lots of conspiracy theories: 
> Hillary Clinton and George Soros are a perfect match, and their agenda appears to be to create a political movement where Soros and his political machine and Clinton are two of the only major players. This is the first time Soros and Clinton have been caught on tape directly colluding in promoting the same false narrative. One of the key revelations in the leaked audio was Clinton's admission to a Russian banker that she knew about the Uranium One deal before it was approved by Congress. Clinton was shown sharing the same talking points that were originally drafted by a Fusion GPS contractor hired by an anti-Trump Republican donor. The leaked audio is the clearest evidence yet that the Clinton campaign and the Hillary Foundation colluded with Fusion GPS to manufacture propaganda against President Trump.


They were concerned enough that they labeled GPT-2 "too dangerous to release", and OpenAI initially refused to release their dataset, training code, or GPT-2 model weights.  OpenAI decided to release in a delayed, phased fashion so that researchers could spend time working on automatic detection of generated text.

In this homework, you'll get to be the judge of how good GPT-2 is, as you'll be using it yourself to generate text!

**To start your journey into the world of Text Generation, you should read Part 1 of the blog post ["The Illustrated GPT-2" by Jay Alammar](http://jalammar.github.io/illustrated-gpt2/) and answer the following questions in 2-3 sentences each**

1. (4 pts) How does the architecture of GPT-2 differ from the standard Encoder-Decoder Transformer model?

<u>The GPT-2 architecture differ because it only uses the decoder layer in the model architecture, and stack multiple of these layers as high as possible depending on the prediction power we need and the amount of data to train.</u>

2. (4 pts) What is the difference between "Masked Self-Attention" and "Self-Attention"

<u>Masked Self-Attention used by GPT-2 decoder, masks the future words tokens to the attention mechanism of the decoder. In this way the present token will connect only to past tokens in the sentence. Instead normal self-attention allows to consider in the attention computation all the tokens in the sentence.</u>

3. (4 pts) What are logits? How are they computed? and How does GPT-2 use them to decide which word to predict next?

<u>Once we receive the output from one decoder unit, we multiply the vector by the embedding matrix that represent all the words in our vocabulary (each row is a word representation). The multiplication between these 2 components results in a list of score probabilities for each word, called logits. The score is the probability of that word being selected as next word prediction. A less smart alternative is to directly choose the highest score word as prediction.</u>

### Aside: GPT-3 

On June 11th 2020, OpenAI released GPT-3 [(paper)](https://arxiv.org/pdf/2005.14165.pdf) [(wikipedia)](https://en.wikipedia.org/wiki/GPT-3). This model has an unfathomable **175 billion parameters** (100x larger than GPT-2!) and was trained on 570GB of text! This model is virtually indistinguishable from human output and can generate text about any topic and in any style with only a few words of priming text. It is by far the largest language model ever trained and it can do some very terrifying things.

GPT-3 Can:
- Generate JSX code off natural language descriptions
- Generate Emojis based off of descriptions of the feeling
- Generate regular expressions off natural language descriptions
- Generate website mockups off natural language descriptions
- Generate charts with titles, labels and legends from natural language descriptions
- Explain python code in plain english
- Automatically generate quiz questions (and grade them)
- Generate Latex from natural language descriptions
- Generate Linux commands from natural language descriptions
- Generate a Machine Learning model from natural language descriptions

[Here's a collection of 21 things GPT-3 can do (with examples)](https://machinelearningknowledge.ai/openai-gpt-3-demos-to-convince-you-that-ai-threat-is-real-or-is-it/#OpenAI_GPT-3_Demos)

[Here's a NYT article about how GPT-3 can write code, poetry, and argue](https://www.nytimes.com/2020/11/24/science/artificial-intelligence-ai-gpt3.html)

[Here's an article GPT-3 wrote for The Guardian about how it loves humans and would never subjugate humanity](https://www.theguardian.com/commentisfree/2020/sep/08/robot-wrote-this-article-gpt-3)

**You may optionally choose to read Jay Alammar's most recent blog post ["How GPT3 Works - Visualizations and Animations"](http://jalammar.github.io/how-gpt3-works-visualizations-animations/) from July 2020 if you're curious as to how GPT-3 differs from GPT-2**

Similarly to GPT-2, OpenAI has decided not to release GPT-3, this time opting to put GPT-3 behind an API which you need to request permission to use. This allows them to control exactly who can generate text and what type of text is generated. While this is a good solution in the short term, the long term implications of GPT-3 are still unclear.

If you are interested in trying out GPT-3 yourself, feel free to [Join the OpenAI API Waitlist](https://share.hsforms.com/1Lfc7WtPLRk2ppXhPjcYY-A4sk30)

-------------------------------

# Part 2: GPT-2 Text Generation with HuggingFace

Phew, that was a lot of reading. Now lets get to the fun part! Let's use the transformer to generate some text!!

We will use the [Transformers library from HuggingFace](https://transformer.huggingface.co), which provides support for many Transformer-based language models like GPT-2. 

**IMPORTANT: Make sure that you have GPU set as your Hardware Accelerator in `Runtime > Change runtime type` before running this Colab.**

In [ ]:
!pip install transformers

## 2.1 The 'Pipeline' Interface

The simplest way to use the HuggingFace library is to use their [Pipeline interface](https://huggingface.co/transformers/main_classes/pipelines.html)

There are many different types of Pipelines available but in this section we'll use the TextGenerationPipeline to get up and running with pretrained gpt2 as fast as possible

In [ ]:
from transformers import pipeline

In [ ]:
# Note: device=0 means to use GPU, device=-1 is to use CPU
generator = pipeline('text-generation', model='gpt2', device=0) 

In [ ]:
outputs = generator('I wonder what I will generate?')
print(outputs)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I wonder what I will generate? I imagine my hand will be full of this thing that comes off of me. What it did to my body, to my mind. What happened to him. It could have been anyone," she says.\n\n'}]


Note that the 'text-generation' pipeline will work with any **auto-regressive** language model (a.k.a 'causal-lm' models according to the HuggingFace lingo). You can find a list of all such models here https://huggingface.co/models?filter=causal-lm. 

(6 pts) **Your first task is to use the Pipeline interface to get generation output below for at least two different 'causal-lm' models (One of these two can be a different version of GPT2, but make sure at least one is a non-gpt family language model)**

In [ ]:
## YOUR CODE HERE FOR MODEL 1
generator = pipeline('text-generation', model='openai-gpt', device=0) 

In [ ]:
outputs = generator('Can I ask one question?')
print(outputs)

In [ ]:
## YOUR CODE HERE FOR MODEL 2
generator = pipeline('text-generation', model='xlnet-large-cased', device=0) 

In [ ]:
outputs = generator("I was in a")
print(outputs)

## 2.2 Dissecting the Pipeline
Now that was easy!

As beautiful and easy as the Pipeline interface is, we want to know what's going on under the hood!

There are four main steps to a text generation pipeline:
1. (Tokenize) Turn the raw input text into a vector of integer token IDs using a tokenizer

2. (Encode) Feed those token IDs into the language model by querying for each token's embedding in the model's embedding matrix (the "encoder") and then feed the "encoded" sequence into the decoder module

3. (Decode) The decoder will output logits (a probability distribution over all possible integer token IDs) and we sample from those logits to get our next token -- repeat until EOS token is generated or we hit max_length

4. (Detokenize) Take the output sequence of token IDs and turn them from integer token IDs back to tokens with the tokenizer

Below you'll see how HuggingFace does this:

First we have to initialize both the tokenizer and the model from their pre-trained checkpoints. Note that the tokenizer has to match the model.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2').cuda()

In [ ]:
#### Step 1: Tokenize the input into integer token IDs
inputs = tokenizer.encode("Hello, how are you?", return_tensors='pt').to(model.device)
print("Input Token IDs: " + str(inputs))

In [ ]:
#### Step 2 and 3: Feed in the integer token IDs and get out a sequence of token IDs as output
outputs = model.generate(inputs)
print("Output Token IDs: " + str(outputs))

In [ ]:
#### Step 4: Feed in the integer token IDs and get out a sequence of token IDs as output
output_text = [tokenizer.decode(x) for x in outputs]
print("Output Text: " + str(output_text))

Now that you have dissected the pipeline, it's time to play with some common parameters!

[Check out this demo notebook from HuggingFace](https://github.com/huggingface/blog/blob/master/notebooks/02_how_to_generate.ipynb) for a good overview of the different generation parameters and what they do (with example code!).

The full documentation on all of the parameters you can use in the generate function can be found [here](https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate)

As an example, below we have a call to generate that:
- randomly samples from the top 50 words in the output distribution (rather than just greedily picking the best one every time)
- downweights the probability of all previously generated tokens by a factor of 1.2 (to prevent repetition)
- goes on for 512 tokens, because its more interesting

In [ ]:
inputs = tokenizer.encode("Hello, how are you?", return_tensors='pt').to(model.device)
outputs = model.generate(
      inputs,
      do_sample=False,          # Randomly sample from the logits instead of greedily picking next word with highest probability
      top_k=50,                 # Only sample from the top 50 most likely words
      repetition_penalty=1.2,    # Downweights the probability of all previously generated tokens by a factor of 1.2
      max_length=512          # Generate for a maximum of 512 tokens
  )
print([tokenizer.decode(x) for x in outputs][0])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, how are you? Why do we need a group in Washington DC?"
A source said of the meeting on Feb. 20: "'The United States Government has put forth an anti-corruption campaign against banks and foreign corporations … These governments want to create money for wealthy individuals at home through tax evasion practices… We have seen people who went into hiding go out there as fast cash." (For more background watch "America First" here.)<|endoftext|>


**Your job is to provide two different examples of generation output from GPT-2 with different choices of generation parameters. You must also provide a 1-2 sentence explanation of what these parameters do and how they affect your output**

Feel free to get creative with this! Really poke around and try to find the combination of settings that gives you the best sounding text! The ways in which these parameters affect how 'human-like' a section of generated text sounds is an area of active research. :)

In [ ]:
## YOUR CODE HERE FOR HYPERPARAMETER VARIATION 1
inputs = tokenizer.encode("Who am I?", return_tensors='pt').to(model.device)
outputs = model.generate(
    inputs,  
    num_beams=5,
    no_repeat_ngram_size=2,
    max_length=512,  
    early_stopping=True
)
print([tokenizer.decode(x) for x in outputs][0])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Who am I?

What do I want to do with my life? How can I make it better? What is the best way to make a difference in the world? Do I need to change my name? Is there something I can do that will make me feel better about myself? If so, what is it about me that makes me so special? And what does it mean to me to be a better person than I am right now?<|endoftext|>


(4 pts) 
<u>
I tested beam search method instead of the sampling randomly. Beams avoid missing some hidden high probabilities while choosing the possible sequence of words. I added the no repetition attribute, to avoid to see multiple time the same word.
</u>

In [ ]:
## YOUR CODE HERE FOR HYPERPARAMETER VARIATION 2
inputs = tokenizer.encode("Who am I?", return_tensors='pt').to(model.device)

sample_outputs = model.generate(
    inputs,
    do_sample=True, 
    max_length=50, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0: Who am I? I was born with an egg, and my parents gave birth to me. I am the daughter of God. It may seem strange to some, but I am the Lord. I believe God's will to me, that I will
1: Who am I? "


What was my purpose in the situation?


"I don't know. There's nothing I've done for you."


"I was not intended to ask questions. This might be for your benefit.
2: Who am I?

I'm not an idiot. I'm a person.

I'm a person because in the beginning I believed it was okay to have children. Now that I've found out, now I can do the same.


(4 pts) 
<u>
I tried another possible sampling method. Instead of using beam search, or simple random sampling, I have experimented a topK and topP sampling strategy, where we consider the top 50 probabilities and from that select the minimum number of words that exceed together the topP porbability to build the sequence.
Also I return multiple output possibilities form the sampling process.
Sampling strategies allows better variation from beam searching, that simulate better the human language.
</u>

## 2.3 Fine-Tuning GPT-2
Okay now time for the best part!

Generating general-purpose text from pre-trained models is great, but what if we want our text to be in a specific genre or style? Luckily for us, the GPT family of models use the idea of "Transfer learning" -- using knowledge gained from one problem (or training setting), and applying it to another area or domain. The idea of transfer learning for NLP, is that we can train a language model on general texts, and then adapt it to use it for a specific task or domain that we're interested in. This process is also called **fine-tuning**.

In this section we'll walk you through an example of using HuggingFace to fine-tune GPT-2 and then you'll be asked to fine-tune GPT-2 on two datasets of your own choosing!

### Fine-Tuning Example using HuggingFace Datasets library: Crime and Punishment

For our fine-tuning example we're going to train GPT-2 to mimic the style of Fyodor Dostoevsky's novel "Crime and Punishment"

We will be downloading our data using the HuggingFace [Datasets](https://huggingface.co/docs/datasets/) library.

In [5]:
!pip install datasets
!pip install transformers

In [6]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, AutoTokenizer, AutoModelForCausalLM
import datasets
from datasets import load_dataset, list_datasets

### Step 1: Initialize a Brand New GPT-2 Model and Tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained('gpt2').cuda()
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

###Step 2: Load the text of "Crime and Punishment" and tokenize it

The 'load_dataset' function queries for a dataset with a certain tag and downloads the corresponding data from HuggingFace's hosting site. This allows us to download all sorts of datasets through the same interface!

The documentation for load_dataset can be found [here](https://huggingface.co/docs/datasets/package_reference/loading_methods.html#datasets.load_dataset)

Here we take our tokenizer and run it on the entirety of Crime and Punishment in a single batch by using map on our custom encode function.

In [8]:
def encode(batch): return tokenizer([x.strip('\n\r') for x in batch['line']], truncation=True, padding=True)

crime_and_punishment = load_dataset('crime_and_punish', split='train')
processed = crime_and_punishment.map(encode, batched=True, batch_size=len(crime_and_punishment))
processed.set_format('torch', columns=['input_ids', 'attention_mask'])

Dataset crime_and_punish downloaded and prepared to /root/.cache/huggingface/datasets/crime_and_punish/crime-and-punish/1.0.0/87ec36ba9cb8741325bea3e40a6d4525210c8f8ef13e7b07872fe32eb72c13ac. Subsequent calls will reuse this data.


### Step 3: Initialize the Trainer

The 'Trainer' module is the main way we perform fine-tuning. In order to initialize a Trainer, you need a model, tokenizer, TrainingArguments, your training data (in a Dataset object) and something called a data_collator (which tells the Trainer not to look for a vector of labels). 

In [9]:
training_args = TrainingArguments(
    output_dir='/content/',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=processed,
)

### Step 4: Fine-Tune the Model!

Now we're done! All we have to do is hit run and sit back!

In [10]:
trainer.train()

Step,Training Loss
100,4.001300
200,3.697400
300,3.662800
400,3.635400
500,3.592000
600,3.591000
700,3.536100
800,3.501100
900,3.507700
1000,3.487800


TrainOutput(global_step=1374, training_loss=3.5788374766165165, metrics={'train_runtime': 366.0166, 'train_samples_per_second': 3.754, 'total_flos': 574101809809920.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': -258048, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 258048, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -159748096, 'train_mem_gpu_alloc_delta': 1499578368, 'train_mem_cpu_peaked_delta': 169816064, 'train_mem_gpu_peaked_delta': 1268570624})

### Step 5: Save the Model and use it to Generate!

Save your fine-tuned model and compare its output with regular GPT-2's output to see the difference for yourself!

In [11]:
trainer.save_model('./dostoevskypt2')

In [12]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, pipeline
dostoevskypt2 = pipeline('text-generation', model='./dostoevskypt2', device=0)
gpt2 = pipeline('text-generation', model='gpt2', device=0)

In [13]:
print(dostoevskypt2('Saint Petersburg is'))
print(gpt2('Saint Petersburg is'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Saint Petersburg is no stranger to such men. One can hardly believe that you, _Pigmatism_. One knows that Porfiry Zagorsky has a great deal in common with Ivan Ilya Petrovitch and that Katerina'}]
[{'generated_text': "Saint Petersburg is home again for a very strong first-ever game and you should make no mistake about that. Their defense is still very good, but you have to be careful with how they run things now, or it's going to get worse."}]


## PERPLEXITY



In [18]:
## FUNCTIONS
!pip install nlp
!pip install transformers
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, AutoTokenizer, AutoModelForCausalLM, AutoModel
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, pipeline
from tqdm import tqdm
from nlp import load_dataset
import torch
device = 'cuda'
model_id = 'gpt2-large'
from google.colab import drive  
drive.mount('/content/drive/')
PATH = '/content/drive/My Drive/Colab/HW5/'

def pp(encodings, model, max_length, model_type='gpt2'):
  #print(max_length)
  stride = 512
  lls = []
  for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
      begin_loc = max(i + stride - max_length, 0)
      end_loc = min(i + stride, encodings.input_ids.size(1))
      trg_len = end_loc - i    # may be different from stride on last loop
      input_ids = encodings.input_ids[:,begin_loc:end_loc].to(device)
      target_ids = input_ids.clone()
      target_ids[:,:-trg_len] = -100
      with torch.no_grad():
          if model_type=='gpt2':
              outputs = model(input_ids, labels=target_ids)
          else:
              outputs = model.model(input_ids, labels=target_ids)
          log_likelihood = outputs[0] * trg_len
      lls.append(log_likelihood)
  ppl = torch.exp(torch.stack(lls).sum() / end_loc)
  print(ppl)

Mounted at /content/drive/


(3 pts) Using the pointer [here](https://huggingface.co/transformers/perplexity.html), compute the perplexity of the GPT2 pre-trained model on the Wikipedia test set (you can keep the same hyperparameters as in the link) 

In [ ]:
## YOUR CODE HERE - FOR COMPUTING PERPLEXITY OF GPT2 ON WIKIPEDIA TEST SET 
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

max_length = model.config.n_positions
test = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
encodings = tokenizer('\n\n'.join(test['text']), return_tensors='pt')
pp(encodings, model, max_length)

Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors
100%|██████████| 562/562 [10:23<00:00,  1.11s/it]


tensor(16.4444, device='cuda:0')


<u>Perplexity score = 16.4444</u>

(2 pts) Compute the perplexity of the dostoevskypt2 model on Wikipedia test set

In [15]:
## PERPLEXITY OF dostoevskypt2 model ON WIKIPEDIA TEST SET 
model = AutoModel.from_pretrained('./dostoevskypt2').cuda()
max_length = model.config.n_positions
dostoevskypt2 = pipeline('text-generation', model='./dostoevskypt2', device=0)
model = dostoevskypt2
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

test = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
encodings = tokenizer('\n\n'.join(test['text']), return_tensors='pt')
pp(encodings, model, max_length, model_type='dosto')

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/8e456126357b4411737ead54576f99321fc077a0d4b64e4a724ab3454ba5b730. Subsequent calls will reuse this data.


Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors
100%|██████████| 562/562 [01:01<00:00,  9.15it/s]


tensor(67.4607, device='cuda:0')


<u> Perplexity score = 67.4607 </u>

(2 pts) Compute the perplexity of the GPT2 pre-trained model on the Crime and Punishment train dataset

In [ ]:
## YOUR CODE HERE - FOR COMPUTING PERPLEXITY OF GPT2 ON CRIME AND PUNISHMENT TRAIN DATASET
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

max_length = model.config.n_positions
test = load_dataset('crime_and_punish', split='train')
encodings = tokenizer('\n\n'.join(test['line']), return_tensors='pt')
pp(encodings, model, max_length)

<u>Perplexity score = 170.9704 </u>

(2 pts) Compute the **train** perplexity of the **dostoevskypt2** model 




In [16]:
## YOUR CODE HERE - FOR COMPUTING PERPLEXITY OF DOSTOEVSKYPT2 ON CRIME AND PUNISHMENT TRAIN DATASET 
model = AutoModel.from_pretrained('./dostoevskypt2').cuda()
max_length = model.config.n_positions
dostoevskypt2 = pipeline('text-generation', model='./dostoevskypt2', device=0)
model = dostoevskypt2
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

test = load_dataset('crime_and_punish', split='train')
encodings = tokenizer('\n\n'.join(test['line']), return_tensors='pt')
pp(encodings, model, max_length, model_type='dosto')

Dataset crime_and_punish downloaded and prepared to /root/.cache/huggingface/datasets/crime_and_punish/crime-and-punish/1.0.0/daee5b1a793a6098a146057861e48d61ca0af622fcacef5514482f93f3ea3d0b. Subsequent calls will reuse this data.


Token indices sequence length is longer than the specified maximum sequence length for this model (360591 > 1024). Running this sequence through the model will result in indexing errors
100%|██████████| 705/705 [01:19<00:00,  8.89it/s]

tensor(61.9602, device='cuda:0')


<u>Perplexity score = 61.9602</u>

(2 pts) Compute the perplexity of the GPT2 model on your Pride and Prejudice text




In [ ]:
## YOUR CODE HERE - FOR COMPUTING PERPLEXITY OF GPT2 ON PRIDE AND PREJUDICE TEXT 
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

file_raw = open(PATH+'prideAndPrejudice.txt', 'r')
test = file_raw.readlines()
file_raw.close()
encodings = tokenizer('\n\n'.join(test), return_tensors='pt')
pp(encodings, model, max_length)

Token indices sequence length is longer than the specified maximum sequence length for this model (155555 > 1024). Running this sequence through the model will result in indexing errors
100%|██████████| 304/304 [03:32<00:00,  1.43it/s]


tensor(17.3736, device='cuda:0')


<u>Perplexity score = 17.3736 </u>

(2 pts) Compute the perplexity of the dostoevskypt2 model on your Pride and Prejudice text




In [19]:
## YOUR CODE HERE - FOR COMPUTING PERPLEXITY OF DOSTOEVSKYPT2 ON PRIDE AND PREJUDICE TEXT 
model = AutoModel.from_pretrained('./dostoevskypt2').cuda()
max_length = model.config.n_positions
dostoevskypt2 = pipeline('text-generation', model='./dostoevskypt2', device=0)
model = dostoevskypt2
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

file_raw = open(PATH+'prideAndPrejudice.txt', 'r')
test = file_raw.readlines()
file_raw.close()
encodings = tokenizer('\n\n'.join(test), return_tensors='pt')
pp(encodings, model, max_length, model_type='dosto')

Token indices sequence length is longer than the specified maximum sequence length for this model (155555 > 1024). Running this sequence through the model will result in indexing errors
100%|██████████| 304/304 [00:32<00:00,  9.22it/s]


tensor(41.2617, device='cuda:0')



<u>Perplexity score = 41.2617 </u>

1. (1 pt) Which model performs better on Wikipedia text?
   > GPT-2 perform better on Wikipedia text (perplexity 16 versus 67)
2. (1 pt) Which model performs better on Pride and Prejudice text?
   > GPT-2 perform better on Pride and Prejudice text (perplexity 17 versus 41)
3. (1 pt) Which model performs better on Crime and Punishment text?
   > Dostoevskypt2 perform better on Crime and Punishment text (perplexity 61 versus 170)

### Now's Your Turn!

**Your job is to fine-tune GPT2 one more time with your choice of fine-tuning dataset.**

*****For the fine-tuned model you create, you should clearly demonstrate (through visible generation outputs and analysis) that your fine-tuned model follows the desired style better than vanilla GPT2** ***

Please make sure to give a brief description 

In order to see which datasets are available for download, run the cell below. Pick one that you think would be interesting!

In [ ]:
datasets_list = list_datasets()
print(', '.join(dataset for dataset in datasets_list))

aeslc, afrikaans_ner_corpus, ag_news, ai2_arc, ajgt_twitter_ar, allegro_reviews, allocine, amazon_reviews_multi, amazon_us_reviews, amttl, anli, arcd, arsentd_lev, art, aslg_pc12, asnq, asset, autshumato, billsum, biomrc, blended_skill_talk, blimp, blog_authorship_corpus, bookcorpus, bookcorpusopen, boolq, break_data, c3, c4, cail2018, cawac, cdsc, cdt, cfq, chr_en, circa, civil_comments, clinc_oos, clue, cmrc2018, cnn_dailymail, coached_conv_pref, coarse_discourse, codah, com_qa, common_gen, commonsense_qa, compguesswhat, conll2000, conll2002, conll2003, conv_ai, coqa, cornell_movie_dialog, cos_e, cosmos_qa, crd3, crime_and_punish, crows_pairs, cs_restaurants, csv, daily_dialog, danish_political_comments, dart, dbpedia_14, deal_or_no_dialog, definite_pronoun_resolution, dialog_re, discofuse, docred, doqa, dream, drop, dyk, e2e_nlg, e2e_nlg_cleaned, eli5, emo, emotion, empathetic_dialogues, eraser_multi_rc, esnli, euronews, event2Mind, evidence_infer_treatment, exams, farsi_news, fever

### Tips
- Most of the datasets hosted by HuggingFace are not meant for Causal LM fine-tuning. Make sure you preprocess them accordingly if you want to use them.
- In order to check out information about a dataset hosted by huggingface you can use [this web viewer](https://huggingface.co/datasets/viewer/?dataset=crime_and_punish). Try to avoid downloading a dataset that's too big!
- You will likely have to change the custom 'encode' function for each new dataset you want to fine-tune on. You need to change batch['line'] to instead index with the correct column label for your specific dataset (it probably wont be called 'line').

### Useful Links
[load_datasets Documentation](https://huggingface.co/docs/datasets/package_reference/loading_methods.html#datasets.load_dataset)

[Trainer Documentation](https://huggingface.co/transformers/main_classes/trainer.html#id1)

[Example: Fine-Tuning BERT for Esperanto](https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=zTgWPa9Dipk2)

[Example: Fine-Tuning for IMDb Classification](https://colab.research.google.com/drive/1-JIJlao4dI-Ilww_NnTc0rxtp-ymgDgM?usp=sharing#scrollTo=5DEWNilys9Ty)


#### Dataset \#1

In [ ]:
## FUNCTIONS
#!pip install datasets
!pip install nlp
!pip install transformers
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, AutoTokenizer, AutoModelForCausalLM, pipeline
#import datasets
#from datasets import load_dataset, list_datasets
from tqdm import tqdm
from nlp import load_dataset
import torch
device = 'cuda'

def encode(batch): return tokenizer([x.strip('\n\r') for x in batch['text']], truncation=True, padding=True)


def pp(encodings, model, max_length, model_type='gpt2'):
  #print(max_length)
  stride = 512
  lls = []
  for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
      begin_loc = max(i + stride - max_length, 0)
      end_loc = min(i + stride, encodings.input_ids.size(1))
      trg_len = end_loc - i    # may be different from stride on last loop
      input_ids = encodings.input_ids[:,begin_loc:end_loc].to(device)
      target_ids = input_ids.clone()
      target_ids[:,:-trg_len] = -100
      with torch.no_grad():
          if model_type=='gpt2':
              outputs = model(input_ids, labels=target_ids)
          else:
              outputs = model.model(input_ids, labels=target_ids)
          log_likelihood = outputs[0] * trg_len
      lls.append(log_likelihood)
  ppl = torch.exp(torch.stack(lls).sum() / end_loc)
  print(ppl)

In [2]:
# Get tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained('gpt2').cuda()
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Preprocessing
#book_corpus = load_dataset('bookcorpus', split='train')
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)
train_dataset = load_dataset('wikitext', 'wikitext-2-v1', split='test')
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=16)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask'])
print(train_dataset)

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/8e456126357b4411737ead54576f99321fc077a0d4b64e4a724ab3454ba5b730. Subsequent calls will reuse this data.



Dataset(features: {'text': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 4358)


In [3]:
# Init and run Trainer
training_args = TrainingArguments(output_dir='/content/',overwrite_output_dir=True, num_train_epochs=1, per_device_train_batch_size=4, per_device_eval_batch_size=4, logging_steps=100, weight_decay=0.01,logging_dir='./logs')
trainer = Trainer(model=model,tokenizer=tokenizer,args=training_args, data_collator=data_collator, train_dataset=train_dataset)
trainer.train()
trainer.save_model('./wikicorpus')

Step,Training Loss
100,3.386900
200,3.076000
300,2.900700
400,3.011400
500,2.930900
600,2.840900
700,2.858000
800,2.724400
900,2.864600
1000,2.840400


(3 pts) YOUR ANSWER HERE - BRIEF DESCRIPTION OF THE DATASET YOU CHOSE

<u> The WikiText language modeling dataset is a collection of over 100 million tokens extracted from the set of verified Good and Featured articles on Wikipedia. I chose the small version of the dataset trained with batch size 4, otherwise the GPU memory was trowing an error of 'full memory'. I tried also the 'imdb' dataset but the structure of the dataset was not suitable for this task (dataset contains a 'label' column, and the train function was trowing an exception;no clear explaination on how to not consider the labels during training of the language model).</u>

In [7]:
## YOUR CODE HERE - FOR GENERATION WITH YOUR FINE-TUNED MODEL AND COMPARISON WITH REGULAR GPT2
dataset_name = 'wikicorpus'
wikicorpus = load_dataset('wikitext', 'wikitext-2-v1', split='train')
encodings = tokenizer('\n\n'.join(wikicorpus['text']), return_tensors='pt')

# Generate
print('\nGenerate with vanilla GPT-2')
gpt2 = pipeline('text-generation', model='gpt2', device=0)
print(gpt2('Today I'))

print('\nGenerate with fine-tuned GPT-2 on wikicorpus')
wikicorpus = pipeline('text-generation', model='./wikicorpus', device=0)
print(wikicorpus('Today I'))


Generate with vanilla GPT-2


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Today I had a meeting with my boyfriend one night and I was like, \'We have to get this together.\' He just said, \'I didn\'t do this to you.\'"\n\n"We had a lot of questions and he said, \''}]

Generate with fine-tuned GPT-2 on wikicorpus


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Today I learned from a conversation with <unk> <unk> , the French <unk> , when he returned to Canada . <unk> was impressed at the French <unk> who had already formed a joint branch with the <unk> .'}]


In [ ]:
## PERPLEXITY
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, pipeline
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
wikicorpus = load_dataset('wikitext', 'wikitext-2-v1', split='train')
encodings = tokenizer('\n\n'.join(wikicorpus['text']), return_tensors='pt')

print('\nPerplexity with vanilla GPT-2 on wikicorpus')
model = GPT2LMHeadModel.from_pretrained('gpt2-large').to(device)
max_length = model.config.n_positions
pp(encodings, model, max_length)

print('\nPerplexity with fine-tuned GPT-2 on wikicorpus')
model = AutoModel.from_pretrained('./wikicorpus').cuda()
max_length = model.config.n_positions
wikicorpus = pipeline('text-generation', model='./wikicorpus', device=0)
model = wikicorpus
pp(encodings, model, max_length, model_type='wiki')

(4 pts) YOUR ANSWER HERE - COMPARISON OF YOUR DATASET'S FINE-TUNED OUTPUT VS NON-FINE-TUNED OUTPUT 

> <u> GENRATED WITH GPT2 (given start 'Today I..'): 

Today I had a meeting with my boyfriend one night and I was like, \'We have to get this together.\' He just said, \'I didn\'t do this to you.\'"\n\n"We had a lot of questions and he said</u>

> <u> GENRATED WITH GPT2 FINETUNED (given start 'Today I..'): 

Today I learned from a conversation with unk unk , the French unk , when he returned to Canada . unk was impressed at the French unk who had already formed a joint branch with the unk .</u>

> <u> PERPLEXITY WITH GPT2 ON THE CORPUS: </u>

20

> <u> PERPLEXITY WITH GPT2 FINETUNED ON THE CORPUS: </u>

15
